In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch

In [2]:
df = pd.read_excel('Kvart_Sovet.xlsx')

In [ ]:
for i in range(len(df['street'])):

    if df['street'][i] == 'Академика Коптюга пр-кт' or df['street'][i] == 'Академика_Коптюга':
        df['street'][i] = 'Академика_Коптюга_проспект'
    elif df['street'][i] == 'Морской пр-кт':
        df['street'][i] = 'Морской_проспект'
    elif df['street'][i] == 'Молодeжи б-р' or df['street'][i] == 'Молодёжи б-р' or df['street'][i] == 'Молодёжи бульвар' or df['street'][i] == 'Молодежи_б-р':
        df['street'][i] = 'Молодежи_бульвар'
    elif df['street'][i] == 'Ученых':
        df['street'][i] = 'Учёных'
    elif df['street'][i] == 'Академика_Иванова':
        df['street'][i] = 'Иванова'

for i in range(len(df['street'])):
    s = ''
    a = df['street'][i].split(' ')

    if len(a) != 1:
        
        for j in range(2):
            if j == 0:
                s += a[j] + '_'
            else:
                s += a[j]

        df['street'][i] = s

In [5]:
df.loc[df.sp_kitchen == '-', 'sp_kitchen'] = np.nan
kvart_new = df[df.sp_kitchen.notna()]

In [6]:
kvart_new = kvart_new[(kvart_new.sp_total < 120) & (kvart_new.price <= 15000)]

In [8]:
kvart_new = kvart_new.dropna(how='all')
kvart_new = kvart_new.drop(columns=['ID', 'city', 'district'])

In [20]:
side_list = []

for i in kvart_new['h_number']:
    try:
        if int(i) % 2 == 0:
            side_list.append(1)
        else:
            side_list.append(0)
            
    except ValueError:
        number = ''

        for k in str(i):
            if k.isdigit() and k != '/':
                number += k
            else:
                break

        if int(number) % 2 == 0:
            side_list.append(1)
        else:
            side_list.append(0)

kvart_new['side'] = side_list
kvart_new = kvart_new.drop(['h_number'], axis=1)

In [77]:
df_dummy_wd = pd.get_dummies(kvart_new, columns=['street', 'rooms'], dtype='float64', drop_first=True)
regressors = df_dummy_wd.drop(columns=['price'], axis=1)
price = df_dummy_wd['price']

In [23]:
train_tensor = main_tensor[:300]
test_tensor = main_tensor[300:380]
valid_tensor = main_tensor[380:]

In [119]:
class RegressionNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(RegressionNet, self).__init__()
        self.fc1 = torch.nn.Linear(67, n_hidden_neurons)
        self.ac1 = torch.nn.Tanh()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 2*n_hidden_neurons)
        self.ac2 = torch.nn.Tanh()
        self.fc3 = torch.nn.Linear(2*n_hidden_neurons, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)

        return x

In [96]:
x_train = torch.tensor(regressors[regressors.columns.to_list()].values.astype(np.float32))[:300]
y_train = torch.tensor(price.values.astype(np.float64))[:300]

x_test = torch.tensor(regressors[regressors.columns.to_list()].values.astype(np.float32))[300:380]
y_test = torch.tensor(price.values.astype(np.float64))[300:380]

x_valid = torch.tensor(regressors[regressors.columns.to_list()].values.astype(np.float32))[380:]
y_valid = torch.tensor(price.values.astype(np.float64))[380:]

In [120]:
net = RegressionNet(300)

In [97]:
x_train.unsqueeze_(1)
y_train.unsqueeze_(1)

x_test.unsqueeze_(1)
y_test.unsqueeze_(1)

x_valid.unsqueeze_(1)
y_valid.unsqueeze_(1)

tensor([[ 2850.],
        [ 3300.],
        [ 1680.],
        [ 2930.],
        [ 2600.],
        [ 1570.],
        [ 4150.],
        [ 6000.],
        [ 5600.],
        [ 2600.],
        [ 6300.],
        [ 4370.],
        [ 2650.],
        [ 2550.],
        [ 2280.],
        [ 2250.],
        [ 3900.],
        [ 2560.],
        [ 2700.],
        [ 1780.],
        [ 2170.],
        [ 2250.],
        [ 3779.],
        [ 2790.],
        [ 3600.],
        [ 5990.],
        [ 3500.],
        [ 1900.],
        [ 2250.],
        [11450.],
        [ 5269.],
        [ 3300.],
        [ 3700.],
        [ 8000.],
        [ 4399.],
        [ 2541.],
        [ 3400.],
        [ 3650.],
        [ 2900.],
        [ 3983.],
        [ 4300.],
        [ 2690.],
        [ 2995.],
        [ 2250.],
        [ 2050.],
        [ 4100.],
        [ 4250.],
        [ 2750.],
        [ 6700.],
        [ 2150.],
        [ 5400.],
        [ 3950.],
        [ 4650.],
        [ 2200.],
        [ 5700.],
        [ 

In [121]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

def loss(pred, target):
    loss = (pred - target)**2
    mean_loss = loss.mean()                             
    return mean_loss

for epoch_index in range(1000):
    optimizer.zero_grad()

    y_pred = net.forward(x_train)
    loss_value = loss(y_pred, y_train)
    loss_value.backward()

    optimizer.step()

# Проверка осуществляется вызовом кода:
def metric(pred, target):
    return (pred - target).abs().mean()

print(metric(net.forward(x_valid), y_valid).item())

2987.7500403619583


In [122]:
y_final_pred = net.forward(x_valid)

compare_df = pd.DataFrame({'Real': y_valid, 'predicted': y_final_pred})

ValueError: If using all scalar values, you must pass an index